<a href="https://colab.research.google.com/github/fernandoGitHub/ML_Projects/blob/main/UNDP_Demographics_Data/UNDP_Demographics_Data_Analysis-Part_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **UNDP_Demographics_Data-Analysis-Part_01**

This notebook will exercise working with several csv files while focusing on cleaning, cleansing and complete missing values using dataframe functionality only

# Setup

In [1]:
import os
import numpy as np
import pandas as pd
import pprint

pp = pprint.PrettyPrinter()

# Data Preparation

In [2]:
import os
import shutil

_RAW_DATA_DIR = './raw_data'
_RAW_DATA_ZIP = os.path.join(_RAW_DATA_DIR, 'UNDP_Demographics_Data.zip')

if os.path.isdir('./sample_data'):
  shutil.rmtree('./sample_data')

if not os.path.isdir(_RAW_DATA_DIR):
  os.makedirs(_RAW_DATA_DIR)

if not os.path.isfile(_RAW_DATA_ZIP):
  !wget -O ./raw_data/UNDP_Demographics_Data.zip https://github.com/fernandoGitHub/ML_Projects/raw/main/UNDP_Demographics_Data/data/UNDP_Demographics_Data.zip

--2022-05-18 10:44:43--  https://github.com/fernandoGitHub/ML_Projects/raw/main/UNDP_Demographics_Data/data/UNDP_Demographics_Data.zip
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/fernandoGitHub/ML_Projects/main/UNDP_Demographics_Data/data/UNDP_Demographics_Data.zip [following]
--2022-05-18 10:44:44--  https://raw.githubusercontent.com/fernandoGitHub/ML_Projects/main/UNDP_Demographics_Data/data/UNDP_Demographics_Data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 44345 (43K) [application/zip]
Saving to: ‘./raw_data/UNDP_Demographics_Data.zip’

./raw_data/UNDP_Dem 100%[===================

In [3]:
_PROCESSED_DATA_DIR = './processed_data'
if not os.path.isdir(_PROCESSED_DATA_DIR):
  os.makedirs(_PROCESSED_DATA_DIR)

In [4]:
# Helper function to clean the raw_data directory
#shutil.rmtree('./raw_data')

In [5]:
from zipfile import ZipFile

zip_file_name = _RAW_DATA_ZIP

with ZipFile(zip_file_name, 'r') as zip:
  # printing all the contents of the zip file
  zip.printdir()

  # extracting all the files
  zip.extractall(_RAW_DATA_DIR)

File Name                                             Modified             Size
Median_age.csv                                 2022-05-13 15:11:54         9288
Old_age_dependency_ratio.csv                   2022-05-13 15:11:54        14159
Population _ages_65 _and _older.csv            2022-05-13 15:11:54        12651
Population_ages_15_64.csv                      2022-05-13 15:11:54        14059
Population_under_age_5.csv                     2022-05-13 15:11:54        12955
Sex_ratio_at_birth.csv                         2022-05-13 15:11:54         9570
Total_Population.csv                           2022-05-13 15:11:54        15523
Urban_Population.csv                           2022-05-13 15:11:54        16114
Young_age_dependency_ratio.csv                 2022-05-13 15:11:54        15756


# Dataset Preparation

First, let's review the columnns at each dataset

In [6]:
def generate_file_list_from_dir(path, filter = '*.*', display=False):
  """generate_list_from_dir(path, filter = '*.*', display=False) returns two lists of strings.
  The first includes the names of the csv files, the second the full paths. The filter parameter
  can be used to return only a given type of files.
  By default, the function doesn't filter any file"""

  file_list = os.listdir(path=path)

  if filter != '*.*':
    file_list = [file for file in file_list if filter in file]

  full_path_list = [os.path.join(path, file) for file in file_list]

  if display:
    print(file_list)

  return file_list, full_path_list


In [7]:
csv_file_list, csv_full_path_list = generate_file_list_from_dir(path=_RAW_DATA_DIR, filter = '.csv', display=False)

for file in csv_full_path_list:
  temp_df = pd.read_csv(file)
  print (f"csv file: {file} - columns: {temp_df.columns}")

csv file: ./raw_data/Old_age_dependency_ratio.csv - columns: Index(['HDI Rank', 'Country', '1990', '1995', '2000', '2005', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
csv file: ./raw_data/Young_age_dependency_ratio.csv - columns: Index(['HDI Rank', 'Country', '1990', '1995', '2000', '2005', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
csv file: ./raw_data/Urban_Population.csv - columns: Index(['HDI Rank', 'Country', '1990', '1995', '2000', '2005', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
csv file: ./raw_data/Population_ages_15_64.csv - columns: Index(['HDI Rank', 'Country', '1990', '1995', '2000', '2005', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
csv file: ./raw_data/Median_age.csv - columns: Index(['HDI Rank', 'Country', 

**Finding:** We can see some irregularities:
   1.- Median and Sex Ratio have less columns (every 5 years)
   2.- Total population has an estimation for 2030

**Next Tasks**
1.   Remove sex-ratio_at_birth from directory (not clear how it helps)
2.   Remove spaces from all CSV files, replace unknown by column mean values and save a new copy
3.   Interpolate values for median dataframe and save to file
4.   Drop the 2030 column from total population as store it as a label dataframe

In [8]:
# Helper Function to remove spaces and replace '..' with -1
def clean(df):
  # We need to remove spaces from the dataframe prior to interpolation
  # And also to replace the unknown values (expressed as two consecutive points)
  # to an arbitrary value of -1
  # Finally we will convert the values to float
  for col in df.columns:
    df[col] = df[col].astype(str).str.strip().replace('..','-1')

  return df

In [9]:
# Helper function to set the columns type
def set_column_type(df, type, inclusive=None, exclusive=None):

  cols = df.columns
  if inclusive == None:
    cols = [col for col in cols if not(col in exclusive)]
  else:
    cols = [col for col in cols if (col in inclusive)]

  for col in cols:
    df[col] = df[col].astype(float)

  return df

In [10]:
# Helper function to replace unknown values by mean values for that year
def replace_unknown_by_mean(df):
  for col in df.columns:
    if df[col].dtype == 'float64':
      mean_value = np.round(df[df[col] != -1][col].mean(), 1)
      df[col] = df[col].replace(-1.0, mean_value)
  
  return df

In [11]:
# Helper function to create interpolated columns
def interpolate_columns(df, new_cols):
  
  def interpolate (y1, y2, x1, x2, x):
    return np.round(y1 + (y2-y1)/(x2-x1) * (x-x1), 1)

  for col in new_cols:
    if not (col in df.columns):
      # Adding a new column and creating the interpolation
      year = int(col)
      prev_year = year - year % 5
      next_year = prev_year + 5

      df[col] = np.vectorize(interpolate)(df[str(prev_year)], df[str(next_year)], prev_year, next_year, year)

  sorted_df = df['Country']
  for col in new_cols:
    sorted_df[col] = df[col]

  return sorted_df

In [12]:
# 1. Removing sex-ratio-at-birth
os.remove('./raw_data/Sex_ratio_at_birth.csv')

csv_file_list, csv_full_path_list = generate_file_list_from_dir(path=_RAW_DATA_DIR, filter = '.csv', display=True)

['Old_age_dependency_ratio.csv', 'Young_age_dependency_ratio.csv', 'Urban_Population.csv', 'Population_ages_15_64.csv', 'Median_age.csv', 'Population_under_age_5.csv', 'Total_Population.csv', 'Population _ages_65 _and _older.csv']


In [13]:
# 2. Remove spaces from all CSV files, replace unknown by column mean values and save a new copy
for file in csv_full_path_list:
  df = pd.read_csv(file)
  df = clean(df)
  df = set_column_type(df, type='float', exclusive=['HDI Rank', 'Country'])
  df = replace_unknown_by_mean(df)
  df = df.set_index('Country')
  df = df.drop('HDI Rank', axis = 1)
  df.to_csv(file, index=True)

In [14]:
# 3. Interpolate values for median_age dataframe and save to file
df = pd.read_csv('./raw_data/Median_age.csv')

new_cols = ['1990', '1995', '2000', '2005', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
df = interpolate_columns(df, new_cols)
df.to_csv('./raw_data/Median_age.csv', index=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [15]:
#4 - Drop the 2030 column from total population as store it as a label dataframe
df = pd.read_csv('./raw_data/Total_Population.csv')
df = df.set_index('Country')

y_pop_2030 = df['2030']
df = df.drop('2030', axis=1)

df.to_csv('./raw_data/Total_Population.csv', index=True)
y_pop_2030.to_csv('./processed_data/y_pop_2030.csv', index=True)

In [16]:
for file in csv_full_path_list:
  temp_df = pd.read_csv(file)
  print (f"csv file: {file} - columns: {temp_df.columns}")

csv file: ./raw_data/Old_age_dependency_ratio.csv - columns: Index(['Country', '1990', '1995', '2000', '2005', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
csv file: ./raw_data/Young_age_dependency_ratio.csv - columns: Index(['Country', '1990', '1995', '2000', '2005', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
csv file: ./raw_data/Urban_Population.csv - columns: Index(['Country', '1990', '1995', '2000', '2005', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
csv file: ./raw_data/Population_ages_15_64.csv - columns: Index(['Country', '1990', '1995', '2000', '2005', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype='object')
csv file: ./raw_data/Median_age.csv - columns: Index(['Country'], dtype='object')
csv file: ./raw_data/Population_under_age_5

**Next Task**
1.   Merge all the dataframes into one and save into the processed_data directory



In [17]:
# 1. Merge all the dataframes into one and save into the processed_data directory
res_df = None
for file in csv_full_path_list:
  df = pd.read_csv(file)
  df = df.set_index('Country')
  cols = df.columns
  new_cols = [col+"-"+os.path.split(file)[1].replace('.csv',"") for col in cols]
  df.columns = new_cols

  if type(res_df) == type(None):
    res_df = df
  else:
    res_df = pd.merge(res_df, df, how='outer', on='Country')

res_df.to_csv('./processed_data/UNDP_Demographics_Data.csv', index=True)